In [1]:
from evaluate import generate_google_link, add_url_column

In [2]:
import pandas as pd 
test = pd.read_excel("C:\\Users\\Spencer\\Desktop\\clean\\data\\data.xlsx")
test.head()

,Company ID,Industry ID,Industry Name,Company Name,Address 1,City,State,Zip,Listing Type,Country,Phone,Google Link
0,2000626178,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,984 State Rt 17,RAMSEY,NJ,07446-1607,Location,United States of America,2.018183e+09,https://www.google.com/maps/place/Anthony's+Co...
1,2000626186,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,80 Parsonage Rd,EDISON,NJ,08837-2416,Location,United States of America,7.327442e+09,https://www.google.com/maps/place/Anthony's+Co...
2,2000626189,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,6401 Jericho Tpke,COMMACK,NY,11725-2832,Location,United States of America,6.318583e+09,https://www.google.com/maps/place/Anthony's+Co...
3,2000626192,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,50 East Wynnewood Road,WYNNEWOOD,PA,19096,Location,United States of America,6.106455e+09,https://www.google.com/maps/place/Anthony's+Co...
4,2000626194,51,Locations: Chain Restaurants,Anthony's Coal Fired Pizza,48 Walkers Brook Dr,READING,MA,01867-3224,Location,United States of America,7.819448e+09,https://www.google.com/maps/place/Anthony's+Co...


In [4]:
test.loc[:, ["Company Name", "Address 1", "City", "State", "Zip"]].apply(generate_google_link, axis=1)[0]

"https://www.google.com/search?q=Anthony's Coal Fired Pizza+984 State Rt 17+RAMSEY+NJ+07446-1607"

In [3]:
add_url_column(test, ["Company Name", "Address 1", "City", "State", "Zip"], "url")

In [1]:
from evaluate import evaluate_locations

In [2]:
data_path = "C:\\Users\\Spencer\\Desktop\\clean\\data\\scraps.csv"
output_path = "C:\\Users\\Spencer\\Desktop\\clean\\data\\review_counts.csv"
id_col = "Company ID"
search_terms = ["Company Name", "Address 1", "City", "State", "Zip"]
new_url_col = "url"

evaluate_locations(data_path=data_path, output_path=output_path, id_col=id_col, search_terms=search_terms, new_url_col=new_url_col)

160 Google reviews
132 Google reviews


WebDriverException: Message: chrome not reachable
  (Session info: chrome=86.0.4240.198)
